## Notebook Covers following topics:
- **Closures Recap**
- **Decorators**
- **Retaining original function doc strings using @wraps**
- **Stacking decorators**
- **Understanding decorators by creating 2 simple decorators and stacking them over a function**
- **Memoization**
    - Understanding difference between using class, closure and decorator approach
    - Making decorator that worked for fibonacci work for factorial also
- **lru_cache() - Inbuilt python memoization decorator**
    - Using maxsize to limit number of elements stored in memory
- **Parametrized decorators**
    - Decorator factory (enables @timed(15) kind of things)
    - How to make a class callable(**call** method)
    - Decorating a class
- **Monkey Patching**
    - Ability to add features to a class after the fact
- **HTMLizing**
- **Single Dispatch** - Inbuilt feature in Python that helps to add features to decorators

In [101]:
#Imports required for this notebook
from functools import wraps, lru_cache, reduce, singledispatch
from time import perf_counter
from datetime import datetime, timezone
from fractions import Fraction
from math import sqrt
from decimal import Decimal
from numbers import Integral
from collections.abc import Sequence

### Closures Recap

***Below is a simple closure that keeps track of count of number of times a function is called***

In [170]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f' Function was called {fn.__name__}, {cnt} number of times')
        return fn(*args, **kwargs)
    return inner

In [171]:
def add(a, b):
    return a + b
def mult(a, b):
    return a * b

In [172]:
counter_add = counter(add)
counter_mult = counter(mult)

In [176]:
counter_add.__closure__

(<cell at 0x0000021CF2832970: int object at 0x00007FFDE3BC3760>,
 <cell at 0x0000021CF28324C0: function object at 0x0000021CF168A3A0>)

In [173]:
counter_add(1,2)
counter_add(1,2)
counter_add(1,2)

 Function was called add, 1 number of times
 Function was called add, 2 number of times
 Function was called add, 3 number of times


3

In [33]:
counter_mult(1,2)
counter_mult(1,2)

 Function was called mult, 1 number of times
 Function was called mult, 2 number of times


2

***Same closure enabled with a dictionary. Dict will keep hold of no: of times function is called***

In [34]:
counter_d = dict()

def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        counter_d[fn.__name__] = cnt
        print(f' Function  called {fn.__name__}, {counter_d[fn.__name__]} number of times')
        return fn(*args, **kwargs)
    return inner

In [35]:
counter_add = counter(add)
counter_mult = counter(mult)

In [36]:
counter_add(1,2)
counter_add(1,2)
counter_add(1,2)
counter_add(1,2)
counter_add(1,2)
counter_d

 Function  called add, 1 number of times
 Function  called add, 2 number of times
 Function  called add, 3 number of times
 Function  called add, 4 number of times
 Function  called add, 5 number of times


{'add': 5}

In [37]:
counter_mult(1,2)
counter_mult(1,2)
counter_mult(1,2)

 Function  called mult, 1 number of times
 Function  called mult, 2 number of times
 Function  called mult, 3 number of times


2

In [38]:
counter_d

{'add': 5, 'mult': 3}

***Same closure modified again to accept a dictionary so that any function that is called can be accounted as a counter in the dictionary***

In [39]:
c = dict()
d = dict()

In [40]:
def counter(fn, ctr_dict):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        ctr_dict[fn.__name__] = cnt
        print(f'Function  called {fn.__name__}, {ctr_dict[fn.__name__]} number of times')
        return fn(*args, **kwargs)
    return inner

In [41]:
counter_add = counter(add, c)
counter_mult = counter(mult, d) # Giving different dictionary for mult. We will later add 'fact' also to this

In [42]:
counter_add(1,2)
counter_add(1,2)
counter_add(1,2)
counter_add(1,2)
counter_add(1,2)
print(f'c: {c}, d: {d}')

Function  called add, 1 number of times
Function  called add, 2 number of times
Function  called add, 3 number of times
Function  called add, 4 number of times
Function  called add, 5 number of times
c: {'add': 5}, d: {}


In [43]:
counter_mult(1,2)
counter_mult(1,2)
counter_mult(1,2)
print(f'c: {c}, d: {d}')

Function  called mult, 1 number of times
Function  called mult, 2 number of times
Function  called mult, 3 number of times
c: {'add': 5}, d: {'mult': 3}


In [44]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [45]:
counter_fact = counter(fact, d) #Using d for counting fact also

In [46]:
counter_fact(1)
counter_fact(2)
counter_fact(6)
counter_fact(10)

Function  called fact, 1 number of times
Function  called fact, 2 number of times
Function  called fact, 3 number of times
Function  called fact, 4 number of times


3628800

In [47]:
counter_fact(16)

Function  called fact, 5 number of times


20922789888000

In [48]:
fact(5)

120

In [50]:
counter_fact(5)

Function  called fact, 6 number of times


120

In [51]:
d

{'mult': 3, 'fact': 6}

***How to set password using closures & then allow to execute a function only after authentication***

In [54]:
def set_password():
    pwd = ''
    def inner():
        nonlocal pwd
        if pwd == '':
            pwd = input()
        return pwd
    return inner

In [55]:
curr_password = set_password()

In [56]:
curr_password.__closure__

(<cell at 0x0000021CF16D4130: str object at 0x0000021CED1722F0>,)

In [57]:
curr_password()  # First time while calling pwd is '', so it will take input & allow to set.

anil


'anil'

In [59]:
curr_password()  # Now pwd is already set as 'anil', so calling it will simply return current pwd

'anil'

In [60]:
## Authenticate

In [61]:
def authenticate(fn, curr_password, user_password):
    cnt = 0
    if user_password == curr_password():
        def inner(*args, **kwargs):
            nonlocal cnt
            cnt += 1
            print(f'fn_name called {cnt} times')
            return fn(*args, **kwargs)
        return inner
    else:
        print('You scamster !!')

In [64]:
counter_add = authenticate(add, curr_password, 'anil')  # Given correct pwd, so allowed to create closure

In [65]:
counter_add(7, 10)

fn_name called 1 times


17

In [66]:
counter_mul = authenticate(add, curr_password, 'ani') # Given incorrect pwd, so didn't allow

You scamster !!


## Decorators

In [73]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'Function  called {fn.__name__}, {cnt} number of times')
        return fn(*args, **kwargs)
    return inner

In [74]:
def add(a, b):
    """
    Adds 2 numbers a & b
    """
    return a + b

In [75]:
add.__doc__

'\n    Adds 2 numbers a & b\n    '

In [76]:
help(add)
id(add)

Help on function add in module __main__:

add(a, b)
    Adds 2 numbers a & b



2323333688048

***Now let us create decorator***

In [77]:
add = counter(add)

In [78]:
add(12,13)

Function  called add, 1 number of times


25

In [79]:
add(7,10)
add(7,10)
add(7,10)

Function  called add, 2 number of times
Function  called add, 3 number of times
Function  called add, 4 number of times


17

***Instead of giving 'add = counter(add)', we can use @counter as shown below***

In [80]:
@counter
def mult(a:int, b:int, c=3, *, d:int):
    "Multiples 4 numbers"
    return a*b*c*d

In [81]:
mult(1,2,4, d=10)

Function  called mult, 1 number of times


80

In [82]:
mult(1,12,4, d=10)

Function  called mult, 2 number of times


480

***Problem is we are losing the identity of original function while decorating it as shown below***

In [84]:
help(add)  #Original add docstring is lost
id(add)    #ID also changed 

Help on function inner in module __main__:

inner(*args, **kwargs)



2323333688192

In [86]:
# same case with mult too
help(mult)  # It is now coming as docstring of 'inner' fucntion defined in our closure
id(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



2323332611376

## @wraps - Decorator factory

***We can solve above pblm as shown below by using @wraps***

In [97]:
from functools import wraps

def counter(fn):
    cnt = 0
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'Function  called {fn.__name__}, {cnt} number of times')
        return fn(*args, **kwargs)
    #inner = wraps(fn)(inner)  --> We can either give this way or using @wraps as above. @wraps is popular method
    return inner

In [98]:
@counter
def mult(a:int, b:int, c=3, *, d:int):
    "Multiples 4 numbers"
    return a*b*c*d

In [99]:
help(mult)
id(mult)

Help on function mult in module __main__:

mult(a: int, b: int, c=3, *, d: int)
    Multiples 4 numbers



2323349570896

In [100]:
mult(1,12,4, d=10)
mult(1,12,4, d=10)
mult(1,12,4, d=10)

Function  called mult, 1 number of times
Function  called mult, 2 number of times
Function  called mult, 3 number of times


480

In [101]:
help(mult)
id(mult)

Help on function mult in module __main__:

mult(a: int, b: int, c=3, *, d: int)
    Multiples 4 numbers



2323349570896

In [112]:
# Let us define a timed function that will tell us how much time a function took to execute
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for k, v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        
        print(f'{fn.__name__}({args_str}) took {elapsed} seconds')
        
        return result
    return inner

In [121]:
# Let us now define a fibonacci function that calculates fibonacci number recursive way & decorate it with 'timed'
@timed
def calc_recursive_fib(n):
    if n <=2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)

In [122]:
calc_recursive_fib(3)

calc_recursive_fib(2) took 8.999995770864189e-07 seconds
calc_recursive_fib(1) took 1.1999945854768157e-06 seconds
calc_recursive_fib(3) took 0.0009948000006261282 seconds


2

***Problem here is log will become huge for bigger numbers as below***

In [123]:
calc_recursive_fib(8)

calc_recursive_fib(2) took 4.00003045797348e-07 seconds
calc_recursive_fib(1) took 6.00004568696022e-07 seconds
calc_recursive_fib(3) took 9.700000373413786e-05 seconds
calc_recursive_fib(2) took 8.999995770864189e-07 seconds
calc_recursive_fib(4) took 0.0003293999980087392 seconds
calc_recursive_fib(2) took 6.00004568696022e-07 seconds
calc_recursive_fib(1) took 1.2000018614344299e-06 seconds
calc_recursive_fib(3) took 0.00012509999942267314 seconds
calc_recursive_fib(5) took 0.0005361000003176741 seconds
calc_recursive_fib(2) took 4.00003045797348e-07 seconds
calc_recursive_fib(1) took 3.9999576983973384e-07 seconds
calc_recursive_fib(3) took 4.579999949783087e-05 seconds
calc_recursive_fib(2) took 3.00002284348011e-07 seconds
calc_recursive_fib(4) took 0.00020640000002458692 seconds
calc_recursive_fib(6) took 0.0008005999989109114 seconds
calc_recursive_fib(2) took 3.9999576983973384e-07 seconds
calc_recursive_fib(1) took 3.9999576983973384e-07 seconds
calc_recursive_fib(3) took 0.0

21

***We can solve this by defining another function and decorate that alone as below***

In [125]:
# This time not decorating with 'timed'
def calc_recursive_fib(n):
    if n <=2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)

In [126]:
# Defining a fuction separately to decorate. This will help to avoid baggage of logs we seen above#
@timed
def fib_recursive(n):
    return calc_recursive_fib(n)

In [127]:
# Now if we call fib_recursive, logs will be minimal
fib_recursive(8)

fib_recursive(8) took 1.1900003300979733e-05 seconds


21

***Let us inspect a more efficient way to calculate fibonacci numbers***

In [132]:
@timed
def fib_loop(n):
    fib_1 = 1
    fib_2 = 1
    for i in range(3, n+1):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2

In [133]:
fib_loop(8)  # We can clearly see this took lesser time

fib_loop(8) took 3.3999967854470015e-06 seconds


21

***Let us try same thing with 'reduce' now***

The ***reduce(fun,seq)*** function is used to apply a particular function passed in its argument to all of the list elements mentioned in the sequence passed along.This function is defined in “functools” module.

Working :  

 - At first step, first two elements of sequence are picked and the result is obtained.
 - Next step is to apply the same function to the previously attained result and the number just succeeding the second element and the result is again stored.
 - This process continues till no more elements are left in the container.
 - The final returned result is returned and printed on console.
 
**Remember lambda function format is -** ***lambda x, y : x + y***

**Below reduce functions works as below**


In [142]:
from functools import reduce

print(reduce(lambda prev,nxt:prev+nxt, range(1), 10000))
print(reduce(lambda prev,nxt:prev+nxt, range(7)))

10000
21


***Now let us use reduce to create a fibonnaci series and decorate it with our timed function***

In [144]:
'''
Explanation of below function

n = 1
(1, 0) --> (1, 1) result t[0] = 1

n = 2
(1, 0) --> (1, 1) --> (2, 1) result t[0] = 2

n = 3
(1, 0) --> (1, 1) --> (2, 1) --> (3, 2) result t[0] = 3


previous value = (a, b)
new value = (a + b, a)
'''

'\nExplanation of below function\n\nn = 1\n(1, 0) --> (1, 1) result t[0] = 1\n\nn = 2\n(1, 0) --> (1, 1) --> (2, 1) result t[0] = 2\n\nn = 3\n(1, 0) --> (1, 1) --> (2, 1) --> (3, 2) result t[0] = 3\n\n\nprevious value = (a, b)\nnew value = (a + b, a)\n'

In [155]:
@timed
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n)
    fib_n = reduce(lambda prev, prev_b4: (prev[0]+prev[1],prev[0]), dummy, initial)
    return fib_n[0]

In [162]:
fib_reduce(5)

fib_reduce(5) took 6.399997801054269e-06 seconds


8

***Now let us enhance the 'timed' closure to capture the total elapsed time when a function is called 'n' times in a single-go. Please note that below closure has that 'limitation'. We have hard-coded 10 here. We need to see how we can overcome this in coming sessions***

In [187]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        total_elapsed_time = 0
        total_elapsed_cnt  = 0 
        
        for i in range(10):  #10 is hard-coded here
            print(f'Running iteration # {i+1}')
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            total_elapsed_time += elapsed
            total_elapsed_cnt  += 1
            
        args_    = [str(a) for a in args]
        kwargs_  = ['{0}={1}' .format(k,v) for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        total_elapsed_avg = total_elapsed_time/total_elapsed_cnt
        
        print(f'{fn.__name__}{all_args} took total {total_elapsed_avg} secs')    
        return result
    return inner

In [188]:
# Now let us decorate 'fib_reduce' function with 'timed'
@timed
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n)
    fib_n = reduce(lambda prev, prev_b4: (prev[0]+prev[1],prev[0]), dummy, initial)
    return fib_n[0]

In [189]:
fib_reduce.__closure__

(<cell at 0x0000021CF2DC7490: function object at 0x0000021CF275EE50>,
 <cell at 0x0000021CF2DC7CD0: builtin_function_or_method object at 0x0000021CED1F3590>)

In [190]:
fib_reduce(10)

Running iteration # 1
Running iteration # 2
Running iteration # 3
Running iteration # 4
Running iteration # 5
Running iteration # 6
Running iteration # 7
Running iteration # 8
Running iteration # 9
Running iteration # 10
fib_reduce['10'] took total 5.789998976979405e-06 secs


89

***Let us see if we can avoide hardcoding "for i in range(10)" by giving like @timed(10) and get rid of hardcoding as below. IT WILL FAIL. So we need to find a way to resolve it. We will see it in coming cells***

In [196]:
def timed(fn, cnt):
    from functools import wraps           #We cant assume that users who call the original function will have these imported
    from time import perf_counter         # Hence keeping these statements inside decorator
    
    @wraps(fn)
    def inner(*args, **kwargs):
        total_elapsed_time = 0
        total_elapsed_cnt  = 0 
        
        for i in range(cnt):  # Changed 10 hard-coded earlier to cnt
            print(f'Running iteration # {i+1}')
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            total_elapsed_time += elapsed
            total_elapsed_cnt  += 1
            
        args_    = [str(a) for a in args]
        kwargs_  = ['{0}={1}' .format(k,v) for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        total_elapsed_avg = total_elapsed_time/total_elapsed_cnt
        
        print(f'{fn.__name__}{all_args} took total {total_elapsed_avg} secs')    
        return result
    return inner

***Trying to give 10 here to see if 'cnt' takes it in 'timed'. It won't work***

In [193]:
@timed(10)       
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n)
    fib_n = reduce(lambda prev, prev_b4: (prev[0]+prev[1],prev[0]), dummy, initial)
    return fib_n[0]

TypeError: timed() missing 1 required positional argument: 'cnt'

***But below will work. But this is not our favorite method. We still need to find a way to make it work with @timed***

In [194]:
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n)
    fib_n = reduce(lambda prev, prev_b4: (prev[0]+prev[1],prev[0]), dummy, initial)
    return fib_n[0]

fib_reduce = timed(fib_reduce, 10)  #Instead of @timed(10)

In [195]:
fib_reduce(7)

Running iteration # 1
Running iteration # 2
Running iteration # 3
Running iteration # 4
Running iteration # 5
Running iteration # 6
Running iteration # 7
Running iteration # 8
Running iteration # 9
Running iteration # 10
fib_reduce['7'] took total 8.64000103319995e-06 secs


21

***Let us write a logged function to keep a log when function is invoked***

In [202]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{fn.__name__} was called at {run_dt}')
        return result
    return inner
        

In [203]:
@logged
def func_1():
    pass

def func_2():
    pass

In [204]:
func_1()

func_1 was called at 2021-06-20 17:34:05.392693+00:00


In [205]:
func_2()

***We can do stacking of decorators as shown below***

In [212]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print(f'{fn.__name__} took {end - start} seconds')
        return result
    return inner

In [213]:
@timed
@logged
def fib_reduce(n):
    initial = (1, 0)
    dummy = range(n)
    fib_n = reduce(lambda prev, prev_b4: (prev[0]+prev[1],prev[0]), dummy, initial)
    return fib_n[0]

In [214]:
fib_reduce(6)

fib_reduce was called at 2021-06-20 17:36:49.660327+00:00
fib_reduce took 7.6299998909235e-05 seconds


13

In [215]:
@timed
@logged
def func_1():
    pass

In [216]:
func_1()

func_1 was called at 2021-06-20 17:36:52.648549+00:00
func_1 took 5.819999933009967e-05 seconds


## Let us make 2 decorators & stack them - This will help to understand how decorators are created

In [220]:
def dec_1(fn):
    def inner():
        print('Dec_1 called')
        return fn()
    return inner

def dec_2(fn):
    def inner():
        print('Dec_2 called')
        return fn()
    return inner

In [221]:
@dec_1
@dec_2
def fn():
    print('Calling fn')

In [222]:
fn()

Dec_1 called
Dec_2 called
Calling fn


## MEMOIZATION

**Let us understand memoization by making our fibonacci function more efficient**

***If we remember recursive way of fibonaaci generation was damn slow. It takes lot of time even for a number like 36 as shown below***

In [225]:
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for k, v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        
        print(f'{fn.__name__}({args_str}) took {elapsed} seconds')
        
        return result
    return inner

def calc_recursive_fib(n):
    if n <=2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)
    
@timed
def fib_recursive(n):
    return calc_recursive_fib(n)

In [226]:
fib_recursive(36)

fib_recursive(36) took 5.66834640000161 seconds


14930352

***Class Approach - Using classes***

In [237]:
class fib:
    def __init__(self):
        self.cache = {1:1, 2:1}
        
    def fib_calc(self, n):
        if n not in self.cache:
            print(f'Calculating fin {n}')
            self.cache[n] = self.fib_calc(n-1) + self.fib_calc(n-2)
        return self.cache[n]  
            

In [240]:
f= fib()

In [241]:
f.fib_calc(10)

Calculating fin 10
Calculating fin 9
Calculating fin 8
Calculating fin 7
Calculating fin 6
Calculating fin 5
Calculating fin 4
Calculating fin 3


55

In [243]:
f.fib_calc(10)  # Please note that it is not calculating again & taking from cache

55

In [244]:
f.fib_calc(12)  # It is calculating only for 11 & 12

Calculating fin 12
Calculating fin 11


144

***Closure approach - Using closures without decorators***

In [295]:
def fib_c():
    
    cache = {1:1, 2:1}
    
    def inner(n):
        if n not in cache:
            print(f'Calculating fib({n})')
            cache[n] = inner(n-1) + inner(n-2)
        return cache[n]
    return inner



In [296]:
f = fib_c()

In [297]:
f(4)

Calculating fib(4)
Calculating fib(3)


3

In [298]:
f(4) # Please note that it is not calculating again & taking from cache

3

In [299]:
f(6) # It is calculating only for  5 & 6

Calculating fib(6)
Calculating fib(5)


8

***Decorator Approach***

- Difference with closure apprach here is that we are passing an external function here

In [306]:
def memorize_fib(fn):
    
    cache = {1:1, 2:1}
    
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner

In [307]:
@memorize_fib
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n - 1) + fib(n-2)

In [308]:
fib(3)

Calculating fib(3)


2

In [309]:
fib(6)         # It is only calculating 6,5,4, for 3 it is taking from memory

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)


8

In [310]:
fib(7)   # It is only calculating 7, rest it is taking from memory

Calculating fib(7)


13

***Let us same for fact as well by making the decorator more generic***

In [311]:
def memorize_fn(fn):
    
    cache = dict()
    
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner

In [312]:
@memorize_fn
def fact(n):
    print(f'Calculating fact({n})')
    return 1 if n < 2 else n * fact(n-1)

In [313]:
fact(4)

Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


24

In [315]:
fact(5)         # It is only calculating 5, rest it is taking from memory

120

## lru_cache()

In [316]:
from functools import lru_cache

@lru_cache()
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n - 1) + fib(n-2)

In [317]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


8

In [318]:
fib(8)

Calculating fib(8)
Calculating fib(7)


21

In [325]:
@lru_cache(maxsize=4)     # maxsize should be power of 2 for efficiency purpose
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n - 1) + fib(n-2)

In [326]:
fib(4)

Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


3

In [327]:
fib(5)          #It took 1,2,3,4 from memory but due to maxsize 1 will get deleted (see below)

Calculating fib(5)


5

In [329]:
fib(2)        # Only 1 removed, 2 stiill in memory

1

In [330]:
fib(1)        # 1 gone from memory because we have given maxsize=4, so when we calculated 5, bottom one(ie 1) got removed

Calculating fib(1)


1

## Parametrized Decorators

***If we remember we were unable to give @timed(15) kind of thing earlier. Parametrized decorators will solve that. Let us see how. But before that a recap***

In [3]:
def timed(fn, reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0

        for i in range(reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (end - start)
        avg_run_time = total_elapsed / reps
        print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_run_time, reps))
        return result
    return inner

def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-2) + calc_fib_recurse(n-1)

def fib(n):
    return calc_fib_recurse(n)

In [4]:
fib = timed(fib, 100)  # Run the fib function 100 times. If called this way, it will work
fib(6)

Avg Run time: 0.000004s (100 reps)


8

In [5]:
# But it wont work as follows which was the problem we faced erlier as well
@timed(100)
def fib(n):
    return calc_fib_recurse(n)

TypeError: timed() missing 1 required positional argument: 'reps'

### Decorator factory

***We can resolve this by using decorator factory i.e. a decorator to wrap a decorator & we will pass parameters through outer decorator***

In [6]:
def dec_factory(n):
    print(f'Running decorator-factory')
    
    def dec(fn):
        print('Running decorator')
        
        def inner(*args, **kwargs):
            print(f'Running inner {n}')
            return fn(*args, **kwargs)
        
        return inner
    
    return dec

In [7]:
decorat_factory = dec_factory('tsai')
decorat_factory

Running decorator-factory


<function __main__.dec_factory.<locals>.dec(fn)>

In [8]:
def my_func():
    print('running my_func')
    
my_func = decorat_factory(my_func)

Running decorator


In [9]:
my_func()

Running inner tsai
running my_func


In [10]:
my_func()

Running inner tsai
running my_func


In [12]:
@dec_factory('anil')
def my_func():
    print('running my_func')

Running decorator-factory
Running decorator


In [13]:
my_func()

Running inner anil
running my_func


In [15]:
# Another example that accepts 2 input parameters

def dec_factory(a, b):
    
    print('Running decorat factory')
    
    def dec(fn):
        print('Running decorator')
        
        def inner(*args, **kwargs):
            
            print('Running inner')
            print(f'a = {a}, b = {b}')
            return fn(*args, **kwargs)
        return inner
    return dec 
        

In [17]:
@dec_factory(10, 20)
def add(x, y):
    return x + y

Running decorat factory
Running decorator


In [18]:
add(100, 200)

Running inner
a = 10, b = 20


300

***Now let us attempt 'timed' with decorator factory. We can successfully pass 'repeat' though decorator factory***

In [21]:
def dec_factory(repeat):
    
    def timed(fn):
        
        from time import perf_counter
        
        def inner(*args, **kwargs):
            total_elapsed = 0
            total_count   = 0
            
            for i in range(repeat):
                
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                
                total_elapsed += (end - start)
                total_count   += 1
                
            average_elapsed =  total_elapsed/total_count
            print(f'{fn.__name__} ran for {average_elapsed} secs {total_count} times')
            return result
        return inner
    return timed                   

In [22]:
@dec_factory(1003)
def add(a, b):
    return a + b

In [23]:
add(10, 20)

add ran for 2.7617148835985466e-07 secs 1003 times


30

In [24]:
@dec_factory(5)
def fib(n):
    return calc_fib_recurse(n)

In [25]:
fib(36)

fib ran for 5.612341960000049 secs 5 times


14930352

***How to make an object callable***

In [33]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, c):
        print(f' a = {self.a}, b = {self.b}, c = {c}')

In [34]:
obj = MyClass()

TypeError: __init__() missing 2 required positional arguments: 'a' and 'b'

In [35]:
obj = MyClass(10, 20)

In [36]:
obj(30)

 a = 10, b = 20, c = 30


***How to decorate a class***

In [44]:
class MyClass:
    def __init__(self, a, b):
        self.a = 'Class Decorator'
        self.b = 'Example'
        
    def __call__(self, fn):
        def inner(*args, **kwargs):
            print(f' Seeing {self.a} {self.b} via - {fn.__name__} example. Arguments supplied are {args} & {kwargs}')
            return fn(*args, **kwargs)
        return inner

In [45]:
@MyClass(10, 20)
def add(a, b):
    return a + b

In [46]:
add(100, 100)

 Seeing Class Decorator Example via - add example. Arguments supplied are (100, 100) & {}


200

### Monkey Patching 

***Simply put, monkey patching is making changes to a module or class while the program is running. Heavily useful in enhancing features of already running programs in production***

In [48]:
from fractions import Fraction

In [49]:
f = Fraction(2,3)
f.denominator

3

In [52]:
#denominator is an attribute already available. We can add new attributes to Fraction as below.
Fraction.purchase_monkeys = 'Purchased Monkey'

***And this gets automatically attached to 'f' object we created earlier from Fraction class***

In [53]:
f.purchase_monkeys

'Purchased Monkey'

In [54]:
# We can define a function & attach it as well
Fraction.purchase_monkeys = lambda self, count: f'Purchased {count} monkeys'

In [55]:
f.purchase_monkeys

<bound method <lambda> of Fraction(2, 3)>

In [56]:
f.purchase_monkeys(10)

'Purchased 10 monkeys'

In [57]:
f

Fraction(2, 3)

In [58]:
f.purchase_monkeys(2/3)

'Purchased 0.6666666666666666 monkeys'

In [64]:
# We can use features already available also while defining function & attaching it. Here we are checking if a given
# fraction is an integer or not
Fraction.is_integral = lambda self: self.denominator == 1

In [60]:
f.is_integral()

False

In [61]:
f

Fraction(2, 3)

In [62]:
f = Fraction(24, 12)
f

Fraction(2, 1)

In [63]:
f.is_integral()

True

***Creating a decorator that can decorate class***

In [13]:
# Let us create the decorator first
def dec_purchase_monkeys_class(cls):
    cls.purchase_monkeys = lambda self, count: f'Class {self.__class__.__name__} purchased {count} monkeys'   
    return cls

In [14]:
# Now let us create a class
class Person:
    pass

In [15]:
#Now let us decorate this class
Person = dec_purchase_monkeys_class(Person)

In [16]:
#Now let us create an object from this class
anil = Person()

In [18]:
#object 'anil' should have purchase_monkeys available & should display message. Let us see
anil.purchase_monkeys(101)

'Class Person purchased 101 monkeys'

***Now let us apply this concept to some more meaningful***

In [31]:
from datetime import datetime, timezone

In [32]:
# Let us create a function that takes an object and returns few items related to when that object is called, its id etc.

def info(obj):
    results = []
    results.append(f'time:{datetime.now(timezone.utc)}')
    results.append(f'Class Name to which the object belongs:{obj.__class__.__name__}')
    results.append(f'ID of object:{hex(id(obj))}')
    for k, v in vars(obj).items():  #vars will give the variables if any in the object
        results.append(f' {k}: {v}')
    return results    

In [33]:
# Now let us create a decorator that can accept a class & call the 'info' function above to give out info

def debug_info(cls):
    cls.debug = info
    return cls

In [34]:
# Now let us create a class and decorate it

@debug_info
class Person:
    def __init__(self, name, country):
        self.name = name
        self.country = country
        
    def msg():
        return f'Hello I am {self.name} from {self.country}'

In [35]:
# Now let us create an object from the class
modi = Person('Narendra Modi', 'India')

In [36]:
# Now let us call debug()
modi.debug()

['time:2021-06-22 09:02:19.092453+00:00',
 'Class Name to which the object belongs:Person',
 'ID of object:0x266bd5dcd48',
 ' name: Narendra Modi',
 ' country: India']

In [37]:
# Let us extend this to a bigger class. 
@debug_info
class Automobile:
    def __init__(self, make, model, year, top_speed):
        self.make      = make
        self.model     = model
        self.year      = year
        self.top_speed = top_speed
        self._speed    = 0 # we want to limit the speed to be in a range
        
    @property
    def speed(self):         # We are using setter because we dont want user to have access to 'top_speed'
        return self._speed
    
    @speed.setter
    def speed(self, new_speed): 
        if new_speed > self.top_speed:
            raise ValueError('Speed Cannot exceed Top Speed')
        else:
            self._speed = new_speed            

In [38]:
car1 = Automobile('Maruti', 'Alto', 2017, 100)

In [40]:
car1.debug()

['time:2021-06-22 09:19:26.870779+00:00',
 'Class Name to which the object belongs:Automobile',
 'ID of object:0x266be01d948',
 ' make: Maruti',
 ' model: Alto',
 ' year: 2017',
 ' top_speed: 100',
 ' _speed: 0']

In [41]:
car1.speed = 255

ValueError: Speed Cannot exceed Top Speed

In [42]:
car1.speed = 80

In [43]:
car1.debug()

['time:2021-06-22 09:19:54.104876+00:00',
 'Class Name to which the object belongs:Automobile',
 'ID of object:0x266be01d948',
 ' make: Maruti',
 ' model: Alto',
 ' year: 2017',
 ' top_speed: 100',
 ' _speed: 80']

***We can combine debug_info & info as a single decorator as shown below***

In [18]:
def debug_info2(cls):
    from datetime import datetime, timezone
    def info(self):
        results = []
        results.append(f'time:{datetime.now(timezone.utc)}')
        results.append(f'Class Name to which the object belongs:{self.__class__.__name__}')
        results.append(f'ID of object:{hex(id(self))}')
        for k, v in vars(self).items():  #vars will give the variables if any in the object
            results.append(f' {k}: {v}')
        return results   
    
    cls.debug = info
    return cls    

In [19]:
# Now let us create a class and decorate it with new decorator

@debug_info2
class Person2:
    def __init__(self, name, country):
        self.name = name
        self.country = country
        
    def msg():
        return f'Hello I am {self.name} from {self.country}'

In [20]:
# Now let us create an object from the class
anil = Person2('Anil Bhatt', 'India')

In [21]:
anil.debug()

['time:2021-06-26 15:27:29.923691+00:00',
 'Class Name to which the object belongs:Person2',
 'ID of object:0x1f6cbbd9d00',
 ' name: Anil Bhatt',
 ' country: India']

***An example for importance of decorators. Instead of creating a method inside class as given below, we can create a decorator for eq & solve the problem***

In [27]:
from math import sqrt

In [32]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f'Point {self.x}, {self.y}'

In [33]:
p1, p2, p3, p4 = Point(1,1), Point(1,0), Point(0,1), Point(1,1)

In [34]:
abs(p1)

1.4142135623730951

In [36]:
p1  # __repr__ in action !

Point 1, 1

In [53]:
p1 == p4   # This coming as false because we didnt define == function

False

In [54]:
# Let us define eq function
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f'Point {x}, {y}'
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

In [56]:
# Let us redefine points again to take 'eq' function as well
p1, p2, p3, p4 = Point(1,1), Point(1,0), Point(0,1), Point(1,1)

In [57]:
p1 == p4

True

In [46]:
# Let us create a decorator
def comple_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not(self < other) and not(self == other)
        cls.__ge__ = lambda self, other: not(self < other)
    return cls

In [50]:
@comple_ordering
class Point2:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f'Point {x}, {y}'
    
    def __eq__(self, other):
        if isinstance(other, Point2):
            return self.x == other.x and self.y == other.y
        else:
            return False
        
    def __lt__(self, other):
        if isinstance(other, Point2):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [51]:
# Let us redefine points again to take 'eq' function as well
p1, p2, p3, p4 = Point2(1,1), Point2(1,0), Point2(0,1), Point2(1,1)

In [52]:
p1 >= p2

True

## HTMLizing

***Escaping is a method that allows us to tell a computer to do something special with the text we supply or to ignore the special function of a character.***

In [58]:
from html import escape

'''
< to &lt;
> to &gt;
& to &amp;
'''

'\n< to &lt;\n> to &gt;\n& to &amp;\n'

In [60]:
# Writing few more conversion functions

def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'  #for an int, gives its hex id also

def html_real(a):
    return f'{round(a, 2)}'         #for a real number round to 2

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')    

def html_list(l):
    items = (f'<li>{html_escape(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f'<li>{k}={v}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_set(arg):
    return html_list(arg)

In [61]:
# Let us see how these work

html_str("""this is 
a multi line string
with special characters: 10 < 1000""")

'this is <br/>\na multi line string<br/>\nwith special characters: 10 &lt; 1000'

In [62]:
print(html_str("""this is 
a multi line string
with special characters: 10 < 1000"""))

this is <br/>
a multi line string<br/>
with special characters: 10 &lt; 1000


In [63]:
print(html_int(255))

255(<i>0xff</i>)


In [64]:
print(html_escape(3 + 10j))

(3+10j)


In [65]:
### But we dont want to write functions like above. Let us see how we can resolve this.

In [66]:
from decimal import Decimal

In [68]:
# One way is to write a function as below

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)

    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)

    elif isinstance(arg, str):
        return html_srt(arg)

    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)

    elif isinstance(arg, dict):
        return html_dict(arg)

    elif isinstance(arg, set):
        return html_set(arg)
    
    else:
        return html_escape(arg)

In [69]:
htmlize((1, 2, 3, 4 ))

'<ul>\n<li>1</li>\n<li>2</li>\n<li>3</li>\n<li>4</li>\n</ul>'

In [70]:
# But it is still cumbersome.So we can make it more simple by using closures

In [71]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real, 
        str: html_str, 
        list: html_list,
        tuple: html_list, 
        set: html_set, 
        dict: html_dict
    }

    fn = registry.get(type(arg), registry[object])
    return fn(arg)

In [72]:
htmlize(100)

'100(<i>0x64</i>)'

In [73]:
print(htmlize([1, 2, 3, 4]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
<li>4</li>
</ul>


In [78]:
# Now let s check decorator approach

def singledispatch(fn):
    registry = {}
    registry[object] = fn
    registry[int] = lambda a: f'{a}(<i>{str(hex(a))}</i>)'
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')

    def inner(arg):
        return registry.get(type(arg), registry[object])(arg)
            # registry.get(type(arg), registry[object]) -> This will be a function like html_str
            # registry.get(type(arg), registry[object])(arg) -> Will be like html_str(arg)   
    
    return inner

In [75]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [76]:
htmlize('1 < 100')

'1 &lt; 100'

In [77]:
# But above approach only have str & int. How can we add more features to the decorator after the fact

## Single Dispatch

In [88]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn

    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)
            # registry.get(type(arg), registry[object]) -> This will be a function like html_str
            # registry.get(type(arg), registry[object])(arg) -> Will be like html_str(arg)
            
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn

        return inner
    
    decorated.register = register
    decorated.registry = registry
    return decorated

In [89]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [90]:
htmlize.register

<function __main__.singledispatch.<locals>.register(type_)>

In [92]:
htmlize.registry # This is now empty

{object: <function __main__.htmlize(a)>}

In [93]:
@htmlize.register(int)
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [83]:
htmlize(100)

'100(<i>0x64</i>)'

In [95]:
htmlize.registry # int got added

{object: <function __main__.htmlize(a)>, int: <function __main__.html_int(a)>}

In [96]:
# Similarly we can register other functions AFTER THE FACT

@htmlize.register(float)
def html_real(a):
    return f'{round(a, 2)}'

from decimal import Decimal
@htmlize.register(Decimal)
def html_real(a):
    return f'{round(a, 2)}'

def html_escape(arg):
    return escape(str(arg))

@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f'<li>{html_escape(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = (f'<li>{k}={v}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [97]:
print(htmlize([1, 2, 3, 4]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
<li>4</li>
</ul>


In [98]:
# let us take a look at registry now

htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 float: <function __main__.html_real(a)>,
 decimal.Decimal: <function __main__.html_real(a)>,
 str: <function __main__.html_str(s)>,
 list: <function __main__.html_sequence(l)>,
 tuple: <function __main__.html_sequence(l)>,
 dict: <function __main__.html_dict(d)>}

In [99]:
htmlize.register

<function __main__.singledispatch.<locals>.register(type_)>

***singledispatch is inbuilt in python***

In [100]:
from functools import singledispatch

In [102]:
from numbers import Integral
from collections.abc import Sequence

In [103]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [104]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>})

In [105]:
## Adding how to handle integral to htmlize

@htmlize.register(Integral)
def htmlize_integral_numbers(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [106]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_numbers(a)>})

In [113]:
# We can also use 'dispatch' to find out the function invoked for a particular type as shown below

htmlize.dispatch(Integral)

<function __main__.htmlize_integral_numbers(a)>